In [1]:
from Discriminamometro import Utileria, Discriminamometro
import pandas as pd
import spacy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier


nlp = spacy.load('./data/tweets/')
# Se instancia la clase del Discriminamometro
obj_Discr = Discriminamometro('PRUEBA')

# Mostramos la cantidad de tweets
print(obj_Discr.nbr_TweetsXCorrida)

8


In [2]:
data = pd.read_csv("data/Tuits Discriminatorios - jul2020 - Muestra Tuits.csv")

In [3]:
data.columns = ['full_text', 'categoria', 'disc', 'Observaciones']

In [4]:
data.head()

,full_text,categoria,disc,Observaciones
0,"!Viva la ""diversidad""! [?] \n\n!Proximamente e...",apariencia,NO,capaz que la imagen es discriminatoria
1,@tennissucios @TenochHuerta Jajajaja... Muy ac...,apariencia,NO,NaN
2,Hay momentos de la vida en los que te das cuen...,apariencia,NO,NaN
3,@hornyboysmex @Hombres_sexgay Hermoso guerito !,apariencia,NO,sextuiter es un mundo raro
4,Que restriccion tiene @DrGiammattei preparado ...,apariencia,NO,NaN


In [5]:
obj_Discr.str_StopWords='StopWords.p'
obj_Discr.CargarStopWords()

data = obj_Discr.LimpiarDataFrame(data)
data = obj_Discr.QuitarStopWordsDataFrame(data)

In [6]:
train, test = train_test_split(data, test_size=0.2)

In [7]:
npEmbeddings = np.empty([0, 300])

for texto in train['full_text']:
    
    #print(texto)
    
    # process a sentence using the model
    doc = nlp(texto)
    
    # print(doc.vector.shape)
    if doc.vector.shape[0]==300:
        npEmbeddings = np.append(npEmbeddings, [doc.vector], axis = 0)
    else:
        npAux = np.empty([1, 300])
        npEmbeddings = np.append(npEmbeddings, npAux, axis = 0)

npEmbeddings.shape

(800, 300)

In [8]:
clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=5, random_state=42).fit(npEmbeddings, train.disc)
clf

/home/francisco/anaconda3/envs/discriminamometro/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=42, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [9]:
npEmbeddings = np.empty([0, 300])
for texto in test['full_text']:
    
    #print(frase)
    
    # process a sentence using the model
    doc = nlp(texto)
    
    # print(doc.vector.shape)
    #print(npEmbeddings.shape)
    #print(doc.vector.shape)
    
    if doc.vector.shape[0]==300:
        npEmbeddings = np.append(npEmbeddings, [doc.vector], axis = 0)
    else:
        npAux = np.empty([1, 300])
        doc = nlp('prueba')
        #npEmbeddings = np.append(npEmbeddings, npAux, axis = 0)
        npEmbeddings = np.append(npEmbeddings, [doc.vector], axis = 0)
        predicted_svm = clf.predict([doc.vector])
        
npEmbeddings.shape

(200, 300)

In [10]:
predicted_svm = clf.predict(npEmbeddings)
np.mean(predicted_svm == test.disc)

0.755

In [16]:
npAux = nlp('cojo rico').vector
predicted_svm = clf.predict([npAux])
predicted_svm

array(['NO'], dtype='<U3')